In [1]:
import sys,os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import torch
import numpy as np
from src.models.dml_rnn import Nuisance_Network, DynamicEffect_estimator
from omegaconf import DictConfig, OmegaConf
from hydra.utils import instantiate
#import lightning trainer
from pytorch_lightning import Trainer
torch.set_default_dtype(torch.double)

c:\Users\mhr_k\miniconda3\envs\R-learner\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
args_dataset = OmegaConf.load('../config/dataset/mimic_synthetic_debug.yaml')
args_model = OmegaConf.load('../config/model/dml_rnn.yaml')
args = OmegaConf.load('../config/config.yaml')
#make args_dataset and args_model a sub dictionary of args_main
args.update(args_dataset)
args.update(args_model)
print(args['dataset'])
print(args.model)

{'_target_': 'src.data.mimic.semi_synthetic_pipeline.MIMICSemiSyntheticDataPipeline', 'seed': 2025, 'name': 'mimic3_synthetic', 'path': 'C:\\Users\\mhr_k\\Data\\mimic_extract\\all_hourly_data_1000.h5', 'min_seq_length': 10, 'max_seq_length': 31, 'max_number': 20, 'data_seed': 2025, 'n_periods': 3, 'n_treatments_seq': 1, 'split': {'val': 0.2, 'test': 0.2}, 'parallel': False, 'te_model': 'sum', 'autoregressive': True, 'vital_list': ['heart rate', 'red blood cell count', 'sodium', 'mean blood pressure', 'systemic vascular resistance', 'glucose', 'chloride urine', 'glascow coma scale total', 'hematocrit', 'positive end-expiratory pressure set', 'respiratory rate', 'prothrombin time pt', 'cholesterol', 'hemoglobin', 'creatinine', 'blood urea nitrogen', 'bicarbonate', 'calcium ionized', 'partial pressure of carbon dioxide', 'magnesium', 'anion gap', 'phosphorous', 'platelets'], 'static_list': ['gender', 'ethnicity', 'age'], 'drop_first': False, 'synth_outcome': {'_target_': 'src.data.mimic.s

In [3]:
data_pipeline = instantiate(args.dataset)
data_pipeline.insert_necessary_args_dml_rnn(args)

c:\Users\mhr_k\OneDrive\Documents\Thesis\Neural-R-Learner\src\data\mimic\load_data.py:234: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  all_vitals = all_vitals.fillna(method='ffill')
c:\Users\mhr_k\OneDrive\Documents\Thesis\Neural-R-Learner\src\data\mimic\load_data.py:235: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  all_vitals = all_vitals.fillna(method='bfill')
100%|██████████| 20/20 [00:02<00:00,  7.74it/s]
c:\Users\mhr_k\OneDrive\Documents\Thesis\Neural-R-Learner\src\data\mimic\semi_synthetic_pipeline.py:120: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  self.all_vitals = self.all_vitals.uns

In [5]:
checkpoint = r'C:\Users\mhr_k\OneDrive\Documents\Thesis\Neural-R-Learner\mlruns\298159040704337493\189a80144c9e49609f1347f476c1f958\artifacts\de-epoch=2-val_loss=0.0000.ckpt'
#First initialize a blank model of dynamic effect estimator,
#  then load the model weights from the checkpoint
de_est = DynamicEffect_estimator(args, data_pipeline)
de_est.load_state_dict(torch.load(checkpoint)['state_dict'])
de_callbacks = []
de_est = DynamicEffect_estimator(args, data_pipeline)
trainer_de = Trainer(
    max_epochs=args.exp.max_epochs_de,
    callbacks=de_callbacks,
    devices=1,
    accelerator='gpu',
    deterministic=True,
    logger = None,
    detect_anomaly=True,
    gradient_clip_val=args.exp.get('gradient_clip_val', 0.0),
    log_every_n_steps=args.exp.get('log_every_n_steps', 50),
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


For hetero settings

In [6]:
from src.utils import compute_gt_individual_dynamic_effects
from torch.utils.data import DataLoader
args.exp['batch_size'] = 2
ind_true_effect = compute_gt_individual_dynamic_effects(args, data_pipeline, subset = 'test')

In [7]:
test_loader = DataLoader(data_pipeline.test_data, batch_size=args.exp.batch_size, shuffle=False, num_workers=1, drop_last=False)
#predict on the test data loader
predictions_de = trainer_de.predict(de_est, test_loader)
predicted_de = torch.cat([pred for pred in predictions_de], dim = 0)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\mhr_k\miniconda3\envs\R-learner\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:219: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting DataLoader 0: 100%|██████████| 2/2 [00:01<00:00,  1.64it/s]


In [11]:
disc_dim, cont_dim = args.dataset.disc_dim, args.dataset.cont_dim
T_intv_disc, T_base_disc = (np.ones((args.dataset.n_periods, disc_dim)), np.zeros((args.dataset.n_periods, disc_dim))) \
                                if disc_dim > 0 else (None, None)
T_intv_cont, T_base_cont = (np.ones((args.dataset.n_periods, cont_dim)), np.zeros((args.dataset.n_periods, cont_dim))) \
                                if cont_dim > 0 else (None, None)
gt_te = data_pipeline.compute_treatment_effect('test', T_intv_disc, T_intv_cont, T_base_disc, T_base_cont)

ConfigAttributeError: Missing key disc_dim
    full_key: dataset.disc_dim
    object_type=dict

In [10]:
predicted_de[0, :5]

tensor([[[-0.1618, -0.1972],
         [ 0.0931, -0.2160],
         [ 0.0279,  0.1654]],

        [[-0.1578, -0.1832],
         [ 0.1028, -0.2161],
         [ 0.0195,  0.1746]],

        [[-0.1693, -0.1774],
         [ 0.1027, -0.2127],
         [ 0.0226,  0.1771]],

        [[-0.1743, -0.1786],
         [ 0.1072, -0.2165],
         [ 0.0138,  0.1757]],

        [[-0.1905, -0.1852],
         [ 0.0997, -0.2346],
         [ 0.0210,  0.1667]]])